In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping
import seaborn as sns
from IPython.core.display_functions import display
from IPython.display import Audio
from livelossplot import PlotLossesKeras

import keras
from keras import layers
from keras.models import Sequential

from keras_preprocessing.image import img_to_array

import tensorflow as tf
from keras.layers import *

import os
from tqdm.notebook import tqdm
from pathlib import Path
import shutil


from music_plots import *
from sklearn.metrics import roc_auc_score
import numpy as np


<h1> Data management </h1>

In [14]:
tracks_df = load("data/tracks.csv")
genres_df = load("data/genres.csv")
features_df = load("data/features.csv")

In [15]:
tracks_df.head()
genres_df.head()
#features_df.head()

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5


In [16]:
track_genres = tracks_df.xs('track', level=0, axis=1)['genre_top'].loc[features_df.dropna().index]
track_genres = track_genres.dropna()
features_df = features_df.loc[track_genres.index]

track_genres

track_id
2         Hip-Hop
3         Hip-Hop
5         Hip-Hop
10            Pop
134       Hip-Hop
           ...   
155315       Rock
155316       Rock
155317       Rock
155318       Rock
155319       Rock
Name: genre_top, Length: 49598, dtype: category
Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken']

In [17]:
# genres dictionary
genres = {}
index = 0
for i in track_genres.unique():
    genres[i] = index
    index += 1
print(genres)

{'Hip-Hop': 0, 'Pop': 1, 'Rock': 2, 'Experimental': 3, 'Folk': 4, 'Jazz': 5, 'Electronic': 6, 'Spoken': 7, 'International': 8, 'Soul-RnB': 9, 'Blues': 10, 'Country': 11, 'Classical': 12, 'Old-Time / Historic': 13, 'Instrumental': 14, 'Easy Listening': 15}


In [18]:
print(len(track_genres))
print(len(features_df))
features_df.isna().any()
features_df

49598
49598


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
155315       -0.319850  0.653044  1.300360 -0.606061 -1.109302 -1.006683   
155316       -0.490129  0.463834  2.321970 -0.084352  1.662914  2.115189   
155317       -0.461559 -0.229601 -0.496632 -0.422033  0.130612 -0.263825   
155318        0.552473 -0.110498 -0.532014  0.263131 -0.224011 -0.530972   
155319       -0.176901  0.187208 -0.050664  0.368843  0.066005 -0.857354   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
155315      0.059643  0.472580 -0.856436 -0.524676  ...  0.108577  0.016631   
155316     -0.237794  5.695442  0.830353  1.951819  ...  0.128410  0.022547   
155317     -0.628103 -0.082687 -0.229483 -0.492753  ...  0.132964  0.023548   
155318      1.713526  1.418444  1.325197  0.120333  ...  0.108324  0.017540   
155319     -0.780860  0.626281 -0.630938 -0.787229  ...  0.088311  0.018328   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
155315      0.014705   6.947788  0.188477  0.037708  0.035156  0.002930   
155316      0.019816   4.448255  0.172852  0.028773  0.028320  0.003906   
155317      0.026527   3.270612  0.196289  0.031116  0.027832  0.002441   
155318      0.020471   2.356727  0.212891  0.038450  0.037109  0.003418   
155319      0.017936   6.188604  0.167480  0.041480  0.038086  0.004883   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
155315      1.764233  0.018679  
155316      0.955388  0.012385  
155317      1.283060  0.019059  
155318      0.828569  0.017904  
155319      1.818740  0.020133  

[49598 rows x 518 columns]

In [19]:
# generate train and test set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# X_train, X_test, y_train, y_test = train_test_split(features_df.iloc[:8000], track_genres.iloc[:8000], test_size=0.4, random_state=42, stratify=track_genres.iloc[:8000])
X_train, X_test, y_train, y_test = train_test_split(features_df, track_genres, test_size=0.4, random_state=42, stratify=track_genres)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

lab_encoder = LabelEncoder()
lab_encoder.fit(y_train)

y_train = lab_encoder.transform(y_train)
y_test = lab_encoder.transform(y_test)

In [20]:
from sklearn.metrics import classification_report

def evaluate_classifier(x_tr, x_te, y_tr, y_te, model):
    model.fit(x_tr, y_tr)
    prediction = model.predict(x_te)
    print(prediction[:10])
    print(y_te[:10])
    proba_train = model.predict_proba(x_te)
    # pd.DataFrame(proba_train, columns=list(np.unique(lab_encoder.inverse_transform(y_train))))
    print(pd.DataFrame(proba_train))
    print(classification_report(y_te, prediction))

<h1>SVM</h1>

In [21]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True)
evaluate_classifier(X_train, X_test, y_train, y_test, svm_model)

[ 5  4  4  6 10  6  4  8  4  5]
[ 5  4  4  6 13  6  8  5  4  5]
             0         1         2         3         4         5         6   \
0      0.000432  0.013030  0.000337  0.000429  0.274691  0.442263  0.061311   
1      0.000239  0.000886  0.000148  0.000509  0.865653  0.014122  0.002083   
2      0.000046  0.000525  0.000063  0.000101  0.896231  0.040369  0.000118   
3      0.001619  0.011390  0.009749  0.000457  0.027445  0.105038  0.617135   
4      0.003051  0.040595  0.002443  0.003884  0.059468  0.415369  0.070893   
...         ...       ...       ...       ...       ...       ...       ...   
19835  0.000038  0.000027  0.001531  0.000006  0.001993  0.005025  0.009321   
19836  0.001157  0.000326  0.000938  0.002344  0.460122  0.010532  0.017377   
19837  0.001245  0.002974  0.109910  0.000892  0.043867  0.096605  0.055557   
19838  0.000146  0.000031  0.000075  0.000032  0.006442  0.016580  0.000112   
19839  0.007263  0.004992  0.000521  0.000307  0.026122  0.554551  

<h1>K-nearest neighbors</h1>

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
evaluate_classifier(X_train, X_test, y_train, y_test, knn_model)

[ 5  4  7  6  5  5  4  5  4 13]
[ 5  4  4  6 13  6  8  5  4  5]
        0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
0      0.0  0.0  0.0  0.0  0.2  0.4  0.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1      0.0  0.0  0.0  0.0  0.6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.4   
2      0.0  0.0  0.0  0.0  0.2  0.0  0.0  0.8  0.0  0.0  0.0  0.0  0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4      0.0  0.2  0.0  0.0  0.0  0.4  0.0  0.0  0.2  0.0  0.2  0.0  0.0  0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
19835  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
19836  0.0  0.0  0.0  0.0  0.2  0.0  0.2  0.0  0.2  0.0  0.0  0.0  0.0  0.4   
19837  0.0  0.0  0.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.0  0.4  0.0   
19838  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
19839  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  

<h1>Random forest</h1>

In [23]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
evaluate_classifier(X_train, X_test, y_train, y_test, rf_model)

[ 5  4  4  6  5  6  4  5  4 13]
[ 5  4  4  6 13  6  8  5  4  5]
         0     1     2    3     4     5     6     7     8     9     10    11  \
0      0.00  0.10  0.00  0.0  0.19  0.41  0.04  0.04  0.02  0.06  0.02  0.01   
1      0.00  0.01  0.01  0.0  0.66  0.09  0.00  0.06  0.01  0.03  0.00  0.00   
2      0.00  0.00  0.00  0.0  0.64  0.14  0.01  0.12  0.01  0.01  0.00  0.00   
3      0.00  0.02  0.01  0.0  0.07  0.15  0.50  0.01  0.02  0.10  0.00  0.03   
4      0.01  0.15  0.00  0.0  0.15  0.30  0.05  0.04  0.07  0.03  0.02  0.00   
...     ...   ...   ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   
19835  0.00  0.00  0.00  0.0  0.01  0.02  0.00  0.00  0.01  0.00  0.00  0.00   
19836  0.00  0.00  0.01  0.0  0.29  0.09  0.03  0.09  0.05  0.05  0.01  0.00   
19837  0.00  0.00  0.00  0.0  0.08  0.26  0.06  0.05  0.05  0.07  0.00  0.00   
19838  0.00  0.00  0.01  0.0  0.05  0.03  0.01  0.04  0.00  0.00  0.00  0.00   
19839  0.00  0.00  0.00  0.0  0.07  0.70  0.00  0.03  0.

/home/lorenzo/Desktop/ML/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lorenzo/Desktop/ML/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lorenzo/Desktop/ML/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h1>Naive bayes</h1>

In [24]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
evaluate_classifier(X_train, X_test, y_train, y_test, nb_model)

[ 9  7 14  8  0  0  9  0 14 15]
[ 5  4  4  6 13  6  8  5  4  5]
                  0              1              2              3   \
0      1.127962e-103   1.172761e-89  7.477034e-115  3.952615e-285   
1       0.000000e+00   0.000000e+00   2.305856e-28   0.000000e+00   
2      6.121847e-139  3.976450e-249   6.677090e-93   0.000000e+00   
3       0.000000e+00  7.459192e-132   0.000000e+00   0.000000e+00   
4       1.000000e+00   1.498967e-78   1.286719e-51  8.605260e-134   
...              ...            ...            ...            ...   
19835   0.000000e+00   0.000000e+00   1.000000e+00   0.000000e+00   
19836   9.855986e-01  1.048532e-174   1.430670e-02   9.689771e-72   
19837   0.000000e+00  2.978627e-186   1.000000e+00   0.000000e+00   
19838   5.747807e-64  2.047683e-268   1.089491e-20  7.039238e-147   
19839   9.796013e-11  3.963807e-163   8.598054e-38  3.379273e-128   

                  4              5              6              7   \
0      6.578748e-122  3.891496e-117   